In [5]:
# Step 1: Import libraries and packages
import os
import glob
import pandas as pd
import numpy as np
import librosa
import torch




In [7]:
os.chdir(os.path.dirname(os.getcwd()))

In [8]:
os.getcwd()

'C:\\Users\\noahc\\VC_Repo'

In [9]:
from models.fake_voice_detection.model import FakeVoiceDetectionModel

ModuleNotFoundError: No module named 'models.fake_voice_detection'

In [ ]:
# Step 2: Load audio files and label them
human_path = os.path.dirname(os.getcwd()) + "/data/raw/cv_delta/clips/*.wav" # change this according to your folder name
ai_path = os.path.dirname(os.getcwd()) + "/data/raw/cv_delta/AIclips/*.wav" # change this according to your folder name

human_files = glob.glob(human_path)
ai_files = glob.glob(ai_path)

human_labels = [0] * len(human_files) # 0 means human voice
ai_labels = [1] * len(ai_files) # 1 means fake voice

audio_files = human_files + ai_files # concatenate two lists of files
audio_labels = human_labels + ai_labels # concatenate two lists of labels

df = pd.DataFrame({"file": audio_files, "label": audio_labels}) # create a dataframe



In [ ]:
# Step 3: Extract features from audio files
def extract_features(file):
    y, sr = librosa.load(file) # load an audio file
    mfccs = librosa.feature.mfcc(y=y, sr=sr) # extract MFCC feature
    mfccs_scaled = np.mean(mfccs.T,axis=0) # take the mean of each column
    
    return mfccs_scaled

features = []

for file in df["file"]:
    features.append(extract_features(file)) # apply the function to each file
    
features = np.array(features) # convert list to array

features = features.reshape(features.shape[0], features.shape[1], 1) # reshape array for model input



In [ ]:
# Step 4: Load pre-trained model and make predictions
model_path = "fake_voice_detection_model.pth" # change this according to your model path

model = FakeVoiceDetectionModel() 
model.load_state_dict(torch.load(model_path)) 
model.eval() 

predictions = []

for feature in features:
    feature_tensor = torch.from_numpy(feature).float() 
    output_tensor = model(feature_tensor.unsqueeze(0)) 
    output_prob = torch.sigmoid(output_tensor).item() 
    output_label = int(output_prob > 0.5) 
    
    predictions.append(output_label)

accuracy = np.mean(predictions == df["label"]) 

print(f"Accuracy: {accuracy}")



In [ ]:
# Step 5: Save or display predictions and labels
output_df = pd.DataFrame({"file": df["file"], "label": df["label"], "prediction": predictions})

output_df.to_csv("output.csv", index=False) 

print(output_df)